# CodeNamesAI Walkthrough

Table of Contents:
* [Downloading Embed Modules](#downloading_embed_modules)
* [Saving Vocabulary Embeddings](#saving_vocabulary_embeddings)
* [Playing As Operative](#playing_as_operative)
* [Playing As SpyMaster](#playing_as_spymaster)
* [Thoughts And Next Steps](#thoughts_and_next_steps)

Feel free to skip around, each section is completely self-contained

<a id="downloading_embed_modules"></a>
# Downloading Embed Modules

The first step is to download a word/sentence embedder from TFHub. These can be very large models {oddly enough, the 'small' version of the Universal Sentence Encoder {1GB} seems to be larger than the 'large' version {800MB}). And they can take a long time to load (closer to hours than minutes) the first time.  Fortunately, once they load once, they are cached on your system, and will load quickly.

In [ ]:
import tensorflow_hub as hub

module_url = "https://tfhub.dev/google/universal-sentence-encoder/2"
# module_url = "https://tfhub.dev/google/Wiki-words-250-with-normalization/1"
# module_url = "https://tfhub.dev/google/Wiki-words-500-with-normalization/1"

# Import the TF Hub module
embed_module = hub.Module(module_url)

<a id="saving_vocabulary_embeddings"></a>
#  Saving Vocabulary Embeddings

First, word embeddings for the vocabulary needs to be made and saved in a lookup table for later reference. This is need for the SpyMasterAI, and needs to include all words that will make up the vocabulary of 'clue' candidates.

Different embeddings need to be saved for each seperate embed_module you intend to use.

In [1]:
from embedder import Embedder
import tensorflow_hub as hub

module_url = "https://tfhub.dev/google/universal-sentence-encoder/2"
# Reduce logging output (otherwise flooded with INFO messages when loading module)
hub.tf.logging.set_verbosity(hub.tf.logging.ERROR)
embed_module = hub.Module(module_url)
vocab1 = 'vocabulary/original.txt'
vocab2 = 'vocabulary/words.txt'
embeddings_path = 'embeddings/USE_embeddings.json'



embedder = Embedder(embed_module, embeddings_path, vocab1)
# Let's see if there was any vocab stored in embeddings_path already
embedder.vocabulary

C:\Users\Micah\Anaconda3\envs\codenames\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


dict_keys([])

In [ ]:
# getting embeddings for all words in the vocab1 vocab list. This could take awhile.
embedder.add_embeddings_from_txt(save=False)

In [ ]:
# adding on the embeddings from vocab2 as well, and saving lookup table
embedder.new_vocabulary_txt = vocab2
embedder.add_embeddings_from_txt(save=True)

You can also manually add words to the embeddings vocabulary, and choose whether or not to write the words to a vocab list.

This could allow for specialized versions of the game where you are only allowed to provide hints from a specialized set of words (like say, disney princesses)

In [ ]:
disney_princess_words = ['Ariel, Jasmine, Aurora, Rapunzel, Belle, Tiana, Cinderella, Merida, Snow, White, Pocahontas, Mulan']

# You can update a pre-existing txt file, but we'll start a new one.
disney_princess_txt = 'vocabulary/disney_princess.txt'
embedder.new_vocabulary_txt = disney_princess_txt

embedder.add_embeddings_from_list(disney_princess_words, add_to_txt=True, save=True)
# this adds to the spymaster vocabulary that includes vocab1 and vocab2.
# To create a specialized, small vocabulary, make a new Embedder object with a new embeddings_path

<a id="playing_as_operative"></a>
#  2) Playing As Operative

As an operative, you can ask the OperativeAI to help you decide which tiles to guess based on a clue.

In [10]:
from operative import OperativeAI
import tensorflow_hub as hub

embed_module = 'blah'

game_words = ['Plane', 'Duck', 'Bat', 'Soldier', 'Pupil',
              'India', 'Germany', 'Tower', 'Bark', 'Litter',
              'March', 'Slug', 'Shot', 'Button', 'Microscope',
              'Fish', 'Back', 'Circle', 'Canada', 'Cell',
              'Slip', 'Triangle', 'Thief', 'Pie', 'Hawk']

op_ai = OperativeAI(embed_module, game_words)

-----
On your first turn, let's say the spymaster gives the clue 'War' for 3 words 
_____

In [ ]:
hint = 'War'
op_ai.recommend_guess(hint)

In [ ]:
# recommendations of 2 words
op_ai.ngram_recommend_guess(hint, n_gram=2)

In [ ]:
# with 25 words, 3 will take awhile (that's a lot of permutations!)
op_ai.ngram_recommend_guess(hint, n_gram=3)


_____
You guess 'Soldier', 'Germany', and 'Shot'. Then, the opposing team guesses 'Thief' and 'Tower' on their turn.

Now, your SpyMaster offers the clue 'Science' for 2.
_____

In [ ]:
# remove words that are no longer in the game
op_ai.remove_words(['Soldier', 'Germany', 'Shot', 'Thief', 'Tower'])

hint = 'Science'
op_ai.recommend_guess(hint)

In [ ]:
# it's that simple!
op_ai.ngram_recommend_guess(hint, n_gram=2, k=10) # you can reduce the number of results shown

<a id="playing_as_spymaster"></a>
# Playing As SpyMaster
Alright, here's where it get's interesting. You want to get help from the SpyMasterAI, which can help you find words that correlate with your teams words, but also try to make sure they don't correlate with the enemy words or the assassin word.

In [35]:
from spymaster import SpyMasterAI
import tensorflow_hub as hub

module_url = ''
embed_module = hub.Module(module_url)

# location of saved embeddings using Embedder in step 1
vocab_embeddings_path = 'blaah'

red_words = ['Bat', 'Pupil', 'India', 'Tower', 'Litter', 'Slug', 'Button', 'Circle']
blue_words = ['Bark', 'Shot', 'Microscope', 'Fish', 'Back', 'Canada', 'Cell', 'Triangle', 'Thief']
nuetral_words = ['Plane', 'Soldier', 'Germany', 'March', 'Slip', 'Pie', 'Hawk']
assasin_word = 'Duck'

spy_ai = SpyMasterAI(embed_module, vocab_embeddings_path, red_words, blue_words, nuetral_words, assasin_word)

{'a': [3, 4, 5], 'b': [6, 4, 6], 'c': [8, 8, 9], 't': [9, 8, 5]}

In [26]:
# blue's turn
spy_ai.recommend_hint('blue')

['a', 'b', 't', 'c']

By default, the SpyMasterAI will try to find the single word with the best semantic similarity to the entire list of team words, represented as a single sentence. In real life a SpyMaster would not generally do this, because trying to generalize so much weakens the semantic connections, making it harder for you team to guess correctly.

A better approach is to feed the AI a smaller set of words which are more likely to have a strong semantic connection:

In [ ]:
spy_ai.recommend_hint('blue', team_words=['Bat', 'Slug', 'Litter', 'Pupil'])
#spy_ai.recommend_hint('blue', team_words=['Tower', 'Litter', 'Button', 'Circle'])


While this is more helpful, you can also feed in the small set of words that you think are likely to have semantic relationships with your hint and chosen words:

In [ ]:
spy_ai.recommend_hint('blue', team_words=['Bat', 'Slug', 'Litter', 'Pupil'], problem_words = ['Bark', 'Duck', 'Fish', 'Cell'])

After you decide on your hint, and you're team makes their guesses, it's red's turn.

In [ ]:
# feed in list if there is more than one word to remove
spy_ai.remove_blue(['Bat', 'Slug'])
spy_ai.remove_neutral('Pie')
# or alternatively:
#spy_ai.remove(['Bat', 'Slug'], 'blue')
#spy_ai.remove(['Bat', 'Slug'], 'neutral')

spy_ai.recommend_hint('red', team_words=['Plane', 'Soldier', 'Germany', 'March'], problem_words = ['India', 'Shot', 'Canada', 'Litter', 'Tower', 'Thief'])

<a id="thoughts_and_next_steps"></a>
# Thoughts And Next Steps

* Game class
* Have AI vs AI game, Operative using Word2Vec and SpyMaster using USE

In [ ]:
# after Game is working, could have it play against itself (only really interesting if word2vec is used in one case and USE in another)